![iceberg-logo](https://www.apache.org/logos/res/iceberg/iceberg.png)

# Iceberg V2 Position Delete Experiments [piyushdubey]

## Get or Create Spark Session

In [151]:
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

# Configure Spark for Iceberg with position delete support
conf = SparkConf()
conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
conf.set("spark.sql.catalog.spark_catalog.type", "hive")
conf.set("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog")
conf.set("spark.sql.catalog.local.type", "hadoop")
conf.set("spark.sql.catalog.local.warehouse", "file:///tmp/warehouse")

# Key settings for position deletes
conf.set("spark.sql.iceberg.delete.mode", "merge-on-read")
conf.set("spark.sql.iceberg.delete.granularity", "file")

spark = SparkSession.builder \
    .appName("IcebergPositionDeletes") \
    .config(conf=conf) \
    .getOrCreate()

spark

25/09/29 20:26:09 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


## Create Database

In [152]:
%%sql

CREATE DATABASE IF NOT EXISTS nyc

25/09/29 20:26:15 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [154]:
## Drop Table

In [155]:
%%sql

DROP TABLE IF EXISTS nyc.taxis

++
||
++
++

In [156]:
## Create Table

In [157]:
%%sql

CREATE TABLE nyc.taxis (
    VendorID bigint,
    tpep_pickup_datetime timestamp,
    tpep_dropoff_datetime timestamp,
    passenger_count double,
    trip_distance double,
    RatecodeID double,
    store_and_fwd_flag string,
    PULocationID bigint,
    DOLocationID bigint,
    payment_type bigint,
    fare_amount double,
    extra double,
    mta_tax double,
    tip_amount double,
    tolls_amount double,
    improvement_surcharge double,
    total_amount double,
    congestion_surcharge double,
    airport_fee double
) USING ICEBERG
TBLPROPERTIES (
    'format-version' = '2',
    'write.delete.mode' = 'merge-on-read',
    'write.delete.granularity' = 'file',
    'write.merge.mode' = 'merge-on-read'
)

++
||
++
++

In [158]:
# Load data from parquet with explicit Iceberg properties
df = spark.read.parquet("/home/iceberg/data/yellow_tripdata_2021-04.parquet")

# Option 1: Use DataFrame API with explicit properties
df.write \
    .format("iceberg") \
    .option("write.delete.mode", "merge-on-read") \
    .option("write.delete.granularity", "file") \
    .option("format-version", "2") \
    .mode("append") \
    .saveAsTable("nyc.taxis")

In [159]:
%%sql

DESCRIBE EXTENDED nyc.taxis

col_name,data_type,comment
VendorID,bigint,None
tpep_pickup_datetime,timestamp,None
tpep_dropoff_datetime,timestamp,None
passenger_count,double,None
trip_distance,double,None
RatecodeID,double,None
store_and_fwd_flag,string,None
PULocationID,bigint,None
DOLocationID,bigint,None
payment_type,bigint,None


In [160]:
%%sql

SELECT COUNT(*) as total_records
FROM nyc.taxis

total_records
2171187


In [161]:
# Verify Table Properties for Position Deletes

In [162]:
%%sql

SHOW TBLPROPERTIES nyc.taxis

key,value
current-snapshot-id,3140210862407448962
format,iceberg/parquet
format-version,2
write.delete.granularity,file
write.delete.mode,merge-on-read
write.merge.mode,merge-on-read
write.parquet.compression-codec,zstd


# See current Snapshot

In [163]:
%%sql

SELECT
    snapshot_id,
    committed_at,
    operation,
    summary,
    manifest_list, 
    parent_id
FROM nyc.taxis.snapshots
ORDER BY committed_at DESC

snapshot_id,committed_at,operation,summary,manifest_list,parent_id
3140210862407448962,2025-09-29 20:26:46.470000,append,"{'engine-version': '3.5.5', 'added-data-files': '1', 'total-equality-deletes': '0', 'app-id': 'local-1759164025940', 'added-records': '2171187', 'total-records': '2171187', 'spark.app.id': 'local-1759164025940', 'changed-partition-count': '1', 'engine-name': 'spark', 'total-position-deletes': '0', 'added-files-size': '33941643', 'total-delete-files': '0', 'iceberg-version': 'Apache Iceberg 1.8.1 (commit 9ce0fcf0af7becf25ad9fc996c3bad2afdcfd33d)', 'total-files-size': '33941643', 'total-data-files': '1'}",s3://warehouse/nyc/taxis/metadata/snap-3140210862407448962-1-2ab5b6cb-83b7-4b48-ba71-e21451f3a8c5.avro,None


In [170]:
# Check data files

In [164]:
%%sql

-- Check data files before deletion

SELECT 
    content, 
    file_path, 
    file_format, 
    record_count
FROM nyc.taxis.files
WHERE content = 0  -- 0 = DATA, 1 = POSITION_DELETES, 2 = EQUALITY_DELETES

content,file_path,file_format,record_count
0,s3://warehouse/nyc/taxis/data/00004-146-8a5378d0-7550-409e-8873-eba1d545133b-0-00001.parquet,PARQUET,2171187


# Delete a few rows

In [165]:
%%sql

DELETE FROM nyc.taxis 
WHERE trip_distance > 1.0 AND trip_distance < 2.5

++
||
++
++

In [169]:
## Delete some more rows

In [166]:
%%sql

DELETE FROM nyc.taxis 
WHERE trip_distance > 5.0 AND trip_distance < 5.5

++
||
++
++

In [167]:
# Verify records were deleted

In [168]:
%%sql

SELECT COUNT(*) as remaining_records FROM nyc.taxis

remaining_records
1219294


In [145]:
# Inspect Position Delete Files

In [148]:
%%sql

SELECT 
    content,
    file_path,
    file_format,
    record_count,
    file_size_in_bytes
FROM nyc.taxis.files
WHERE content = 1  -- 1 = POSITION_DELETES

content,file_path,file_format,record_count,file_size_in_bytes
1,s3://warehouse/nyc/taxis/data/00000-125-54f5ec8b-9ac6-4e02-9953-1a416655bdeb-00001-deletes.parquet,PARQUET,262536,294761


In [137]:
# Get Deleted Files Path

In [142]:
%%sql
    
SELECT 
    file_path 
FROM demo.nyc.taxis.files LIMIT 20;

SELECT 
    content, 
    file_path 
FROM demo.nyc.taxis.delete_files;

DESCRIBE TABLE EXTENDED demo.nyc.taxis;

ParseException: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'SELECT': extra input 'SELECT'.(line 6, pos 0)

== SQL ==
    
SELECT 
    file_path 
FROM demo.nyc.taxis.files LIMIT 20;

SELECT 
^^^
    content, 
    file_path 
FROM demo.nyc.taxis.delete_files;

DESCRIBE TABLE EXTENDED demo.nyc.taxis;
